In [11]:
import pandas as pd
from operator import itemgetter
import csv
import numpy as np
df = pd.read_csv (r"../data/raw_labelbox_exports/export-2019-10-18T15_02_36.044Z.csv")

### Using this to get the entries with no consensus

### remove unnecessary data

In [12]:
df.sort_values(by=["External ID"], inplace=True)
df = df.reset_index() 
for index, row in df.iterrows():
    if row["Label"] == "{\"leadership_spectrum\":\"3\"}":
        df.iloc[index, 4] = "IDK"
    if row["Label"] == "{\"leadership_spectrum\":\"2\"}":
        df.iloc[index,4] = "STC"
    if row["Label"] == "{\"leadership_spectrum\":\"1\"}":
        df.iloc[index, 4] = "TC"
    if row["Label"] == "{\"leadership_spectrum\":\"0\"}":
        df.iloc[index, 4] = "NA"
    if row["Label"] == "{\"leadership_spectrum\":\"4\"}":
        df.iloc[index, 4] = "STF"
    if row["Label"] == "{\"leadership_spectrum\":\"5\"}":
        df.iloc[index, 4] = "TF"
    if row["Created By"] == "dnsmith@g.clemson.edu":
        df.iloc[index, 5] = "b"
    if row["Created By"] == "ajames8@g.clemson.edu":
        df.iloc[index, 5] = "a"
    if row["Created By"] == "dcottle@g.clemson.edu":
        df.iloc[index, 5] = "c"
    if row["Created By"] == "mkollur@g.clemson.edu":
        df.iloc[index, 5] = "d"
df.head()

,index,ID,DataRow ID,Labeled Data,Label,Created By,Project Name,Created At,Updated At,Seconds to Label,External ID,Agreement,Benchmark Agreement,Benchmark ID,Benchmark Reference ID,Dataset Name,Reviews,View Label
0,596,ck18p4z57mwhv0863z7wgkabn,ck0zz58quomb40bo43bp5bc0g,"""1 - Ability to learn new technology quickly 2...",NA,d,Career - Fall 2019 All Labels,1569986016000,1570196613000,5.205,0_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...
1,817,ck1c6isc9q65j0794ol1c2fi5,ck0zz58quomb40bo43bp5bc0g,"""1 - Ability to learn new technology quickly 2...",NA,a,Career - Fall 2019 All Labels,1570196613000,1570196613000,6.152,0_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...
2,404,ck18d6trtvxee0a468wskapqf,ck0zz58quomb40bo43bp5bc0g,"""1 - Ability to learn new technology quickly 2...",NA,c,Career - Fall 2019 All Labels,1569965947000,1570196613000,9.550,0_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...
3,489,ck18nc7o2oo7j0944wgthijdz,ck0zz58quomb40bo43bp5bc0g,"""1 - Ability to learn new technology quickly 2...",NA,b,Career - Fall 2019 All Labels,1569982995000,1570196613000,5.928,0_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...
4,1668,ck1smh34trxe00721kj2w2q00,ck0zz58quomb50bo4d16bfisw,3 - Dependable and ability to work independent...,NA,d,Career - Fall 2019 All Labels,1571190866000,1571234699000,10.098,0_1,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...


In [13]:
df.groupby("Label").agg("count")

,index,ID,DataRow ID,Labeled Data,Created By,Project Name,Created At,Updated At,Seconds to Label,External ID,Agreement,Benchmark Agreement,Benchmark ID,Benchmark Reference ID,Dataset Name,Reviews,View Label
Label,,,,,,,,,,,,,,,,,
IDK,197,197,197,197,197,197,197,197,197,197,192,0,0,0,197,197,197
NA,1501,1501,1501,1501,1501,1501,1501,1501,1501,1501,1429,0,0,0,1501,1501,1501
STC,72,72,72,72,72,72,72,72,72,72,70,0,0,0,72,72,72
STF,131,131,131,131,131,131,131,131,131,131,128,0,0,0,131,131,131
Skip,25,25,25,25,25,25,25,25,25,25,25,0,0,0,25,25,25
TC,31,31,31,31,31,31,31,31,31,31,30,0,0,0,31,31,31
TF,72,72,72,72,72,72,72,72,72,72,72,0,0,0,72,72,72


### Assign final labels

In [14]:
df.sort_values(by=['index'])


# indices of labels: 0 = NA, 1 = TC, 2 = STC, 3 = IDK, 4= STF, 5 = TF
styles = {
    "IDK" : 0,
    "NA" : 1,
    "STC": 2,
    "STF": 3,
    "Skip": 4,
    "TC" : 5,
    "TF":6
}

# labels[0] is the label, label[1] is the count, label[2] is the most authoritative selection
labels = [["IDK", 0, "z"],["NA", 0, "z"],["STC", 0, "z"],["STF", 0, "z"],["Skip", 0, "z"],["TC", 0, "z"],["TF", 0, "z"]] 
next_id = ""

#add column for final label
df["Final Label"] = ""

for i in range(len(df.index)):
    label = df.iloc[i, 4]
    person = df.iloc[i, 5]
    curr_id = df.iloc[i, 2]
    next_id = "" if i >= len(df.index)-1 else df.iloc[i+1, 2]
    agreement = df.iloc[i,11]
    
    
    if label == "Skip" or label == "{}":
        continue
    else:
        labels[styles[label]][1] += 1

    #keeps track of the "lowest" person, to find most authoritative
    if person < labels[styles[label]][2] or labels[styles[label]][2] == "z":
        labels[styles[label]][2] = person
            

    if curr_id != next_id :

        labels.sort(key=itemgetter(1), reverse=True)
        if(labels[0][1] == labels[1][1]):
         
            labels.sort(key = itemgetter(2))

    
        df.iloc[i,18] = labels[0][0]         
    
        for j in labels:
            j[1] = 0
            j[2] = "z"
        #the below works because this whole cell of code depends on the sorting IDK < NA < TC < TF
        labels.sort(key=itemgetter(0))


In [15]:
df.head()

,index,ID,DataRow ID,Labeled Data,Label,Created By,Project Name,Created At,Updated At,Seconds to Label,External ID,Agreement,Benchmark Agreement,Benchmark ID,Benchmark Reference ID,Dataset Name,Reviews,View Label,Final Label
0,596,ck18p4z57mwhv0863z7wgkabn,ck0zz58quomb40bo43bp5bc0g,"""1 - Ability to learn new technology quickly 2...",NA,d,Career - Fall 2019 All Labels,1569986016000,1570196613000,5.205,0_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,
1,817,ck1c6isc9q65j0794ol1c2fi5,ck0zz58quomb40bo43bp5bc0g,"""1 - Ability to learn new technology quickly 2...",NA,a,Career - Fall 2019 All Labels,1570196613000,1570196613000,6.152,0_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,
2,404,ck18d6trtvxee0a468wskapqf,ck0zz58quomb40bo43bp5bc0g,"""1 - Ability to learn new technology quickly 2...",NA,c,Career - Fall 2019 All Labels,1569965947000,1570196613000,9.550,0_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,
3,489,ck18nc7o2oo7j0944wgthijdz,ck0zz58quomb40bo43bp5bc0g,"""1 - Ability to learn new technology quickly 2...",NA,b,Career - Fall 2019 All Labels,1569982995000,1570196613000,5.928,0_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NA
4,1668,ck1smh34trxe00721kj2w2q00,ck0zz58quomb50bo4d16bfisw,3 - Dependable and ability to work independent...,NA,d,Career - Fall 2019 All Labels,1571190866000,1571234699000,10.098,0_1,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,


In [16]:
df.loc[:,"Labeled Data"] = df.loc[:,"Labeled Data"].str.replace('"', "")
df["Final Label"].replace("", np.nan, inplace=True)
df.head()

,index,ID,DataRow ID,Labeled Data,Label,Created By,Project Name,Created At,Updated At,Seconds to Label,External ID,Agreement,Benchmark Agreement,Benchmark ID,Benchmark Reference ID,Dataset Name,Reviews,View Label,Final Label
0,596,ck18p4z57mwhv0863z7wgkabn,ck0zz58quomb40bo43bp5bc0g,1 - Ability to learn new technology quickly 2 ...,NA,d,Career - Fall 2019 All Labels,1569986016000,1570196613000,5.205,0_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NaN
1,817,ck1c6isc9q65j0794ol1c2fi5,ck0zz58quomb40bo43bp5bc0g,1 - Ability to learn new technology quickly 2 ...,NA,a,Career - Fall 2019 All Labels,1570196613000,1570196613000,6.152,0_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NaN
2,404,ck18d6trtvxee0a468wskapqf,ck0zz58quomb40bo43bp5bc0g,1 - Ability to learn new technology quickly 2 ...,NA,c,Career - Fall 2019 All Labels,1569965947000,1570196613000,9.550,0_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NaN
3,489,ck18nc7o2oo7j0944wgthijdz,ck0zz58quomb40bo43bp5bc0g,1 - Ability to learn new technology quickly 2 ...,NA,b,Career - Fall 2019 All Labels,1569982995000,1570196613000,5.928,0_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NA
4,1668,ck1smh34trxe00721kj2w2q00,ck0zz58quomb50bo4d16bfisw,3 - Dependable and ability to work independent...,NA,d,Career - Fall 2019 All Labels,1571190866000,1571234699000,10.098,0_1,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NaN


In [17]:
df = df.dropna(subset=['Final Label'], axis=0).reset_index(drop=True)
#df = df.drop_duplicates(subset=['DataRow ID'],keep = 'first')
df.head()

,index,ID,DataRow ID,Labeled Data,Label,Created By,Project Name,Created At,Updated At,Seconds to Label,External ID,Agreement,Benchmark Agreement,Benchmark ID,Benchmark Reference ID,Dataset Name,Reviews,View Label,Final Label
0,489,ck18nc7o2oo7j0944wgthijdz,ck0zz58quomb40bo43bp5bc0g,1 - Ability to learn new technology quickly 2 ...,NA,b,Career - Fall 2019 All Labels,1569982995000,1570196613000,5.928,0_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NA
1,1904,ck1tckkz37nbb0721rzq4ewd9,ck0zz58quomb50bo4d16bfisw,3 - Dependable and ability to work independent...,NA,c,Career - Fall 2019 All Labels,1571234699000,1571234699000,7.581,0_1,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NA
2,245,ck18bbur1icot0838zya823hf,ck0zz58r0omgd0bo44vxne51z,Corey can work on planning out the next phases...,NA,b,Career - Fall 2019 All Labels,1569962823000,1570196344000,41.939,100_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NA
3,1803,ck1t4bu8gxca00838w16v9fz0,ck0zz58r0omge0bo42r2lemdp,This skill will come with experience as he dev...,NA,a,Career - Fall 2019 All Labels,1571220854000,1571234380000,2.436,100_1,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NA
4,416,ck18dbc2gixv20838esfz4wqg,ck0zz58r0omgg0bo4f2fade49,Courtney's unfamiliarity with the field kept h...,NA,c,Career - Fall 2019 All Labels,1569966158000,1570196705000,14.719,102_0,1.0,NaN,NaN,NaN,mentor_responses_sentences_up_to_f19.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NA


In [21]:
df_clean = df.loc[:,['Labeled Data', 'Final Label']]
df_clean.to_csv(r"../data/data_for_model/labelbox_clean_fall2019.csv", index=False, quoting = csv.QUOTE_NONNUMERIC)

### Trying to make a csv file for the process_for_watson script

In [123]:
df_clean.to_csv(r"../data/data_for_model/labelbox_v2-1_watson.csv", index=False, sep='|')